<a href="https://colab.research.google.com/github/shnhrtkyk/JTCcode/blob/main/02_%E7%89%B9%E5%BE%B4%E7%82%B9%E3%83%BB%E7%89%B9%E5%BE%B4%E9%87%8F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 特徴点・特徴量

## ライブラリのインストール

In [ ]:
!pip install open3d==0.16

## データのダウンロード
点群解析の分野で広く用いられるスタンフォードバニーと呼ばれるうさぎの点群を用います

In [ ]:
!wget http://graphics.stanford.edu/pub/3Dscanrep/bunny.tar.gz
!tar xvfz ./bunny.tar.gz

## 特徴点
特徴点の算出にはHariis3Dと呼ばれるアルゴリズムを用います。

### Hariis3D

Hariis3DはOpen3Dに関数が用意されていないため、Numpyで実装します。
Hariis3Dでを特徴点を抽出する関数は`compute_harris3d_keypoints`です。


In [ ]:
import sys
import open3d as o3d
import numpy as np


# この関数は特徴点を可視化するために球を作成します
def keypoints_to_spheres(keypoints):
    spheres = o3d.geometry.TriangleMesh()
    for keypoint in keypoints.points:
        sphere = o3d.geometry.TriangleMesh.create_sphere(radius=0.005)
        sphere.translate(keypoint)
        spheres += sphere
    spheres.paint_uniform_color([1.0, 1.0, 1.0])
    return spheres

# Harris3Dを計算する関数
# 引数には，pcd:点群，radius: 探索する半径,  max_nn: 近傍点の数, threshold: 閾値
def compute_harris3d_keypoints(pcd, radius=0.01, max_nn=10, threshold=0.001 ):
    # まず，各点の法線を求めます
    pcd.estimate_normals(
        search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=radius, max_nn=max_nn))
    pcd_tree = o3d.geometry.KDTreeFlann(pcd)
    harris = np.zeros( len(np.asarray(pcd.points)) )
    is_active = np.zeros( len(np.asarray(pcd.points)), dtype=bool )

    # 次に，Harris3Dを求めます
    for i in range( len(np.asarray(pcd.points)) ):
        # 各点の最近傍の点を検索します
        [num_nn, inds, _] = pcd_tree.search_knn_vector_3d(pcd.points[i], max_nn)
        # 最近傍の点のみを解析対象にする
        pcd_normals = pcd.select_by_index(inds)
        pcd_normals.points = pcd_normals.normals
        # 法線の平均値と共分散を計算する
        [_, covar] = pcd_normals.compute_mean_and_covariance()
        # 平均値と共分散に基づいて，Harris3Dを計算する
        harris[ i ] = np.linalg.det( covar ) / np.trace( covar )
        if (harris[ i ] > threshold):
            is_active[ i ] = True

    # ノンマキシマムサプレッションで過剰検出を抑制
    for i in range( len(np.asarray(pcd.points)) ):
        if is_active[ i ]:
            [num_nn, inds, _] = pcd_tree.search_knn_vector_3d(pcd.points[i], max_nn)
            inds.pop( harris[inds].argmax() )
            is_active[ inds ] = False

    keypoints = pcd.select_by_index(np.where(is_active)[0])
    return keypoints

# ファイル名の定義，今回はcolab上にダウンロードしたスタンフォードバニーを用います
filename = "bunny/reconstruction/bun_zipper.ply"
print("Loading a point cloud from", filename)
# oprn3dの読み込み関数を利用
pcd = o3d.io.read_point_cloud(filename)
print(pcd)
# 特徴点取得関数の呼び出し
keypoints = compute_harris3d_keypoints( pcd )
print(keypoints)
# 可視化を行います
pcd.paint_uniform_color([0.5, 0.5, 0.5])
o3d.visualization.draw_plotly(
  [keypoints_to_spheres(keypoints), pcd],
  width=1200,
  height=800,
  front=[ 0.0, 0.0, 1.0], # 表示位置の設定
  lookat=[ -0.016840399999999998, 0.11015419999999999, -0.0015369499999999987],   # 表示位置の設定
  up=[0.0, 1.0, 0.0]     # 表示位置の設定
)


### ISS

ISSはOpen3Dに用意されており、`compute_iss_keypoints`を用います。



In [ ]:
import sys
import open3d as o3d

# この関数は特徴点を可視化するために球を作成します
def keypoints_to_spheres(keypoints):
    spheres = o3d.geometry.TriangleMesh()
    for keypoint in keypoints.points:
        sphere = o3d.geometry.TriangleMesh.create_sphere(radius=0.005)
        sphere.translate(keypoint)
        spheres += sphere
    spheres.paint_uniform_color([1.0, 1.0, 1.0])
    return spheres

# ファイル名の定義，今回はcolab上にダウンロードしたスタンフォードバニーを用います
filename = "bunny/reconstruction/bun_zipper.ply"
print("Loading a point cloud from", filename)
# oprn3dの読み込み関数を利用
pcd = o3d.io.read_point_cloud(filename)
print(pcd)

# 特徴点の取得をopen3dで用意されている関数を用いて実行
keypoints = \
o3d.geometry.keypoint.compute_iss_keypoints(pcd)
print(keypoints)
# 法線の推定
pcd.estimate_normals(
    search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.01, max_nn=10))
pcd.paint_uniform_color([0.5, 0.5, 0.5])
pcd.paint_uniform_color([0.5, 0.5, 0.5])
o3d.visualization.draw_plotly(
  [keypoints_to_spheres(keypoints), pcd],
  width=1200,
  height=800,
  front=[ 0.0, 0.0, 1.0], # 表示位置の設定
  lookat=[ -0.016840399999999998, 0.11015419999999999, -0.0015369499999999987],   # 表示位置の設定
  up=[0.0, 1.0, 0.0]     # 表示位置の設定
)


## 特徴量
特徴量の作成には，点ごとの特徴に着目した局所特徴量と，点群全体の特徴に着目した大域特徴量に分かれます

## 局所特徴量
まずは局所特徴量について説明します
点群の局所特徴量は、画像のマッチングに使用される画素の特長量と同様の考え方です。
違う角度から取得された点群どうしのマッチングに利用可能な点の特徴量を計算します．


### FPFH


FPFHは、Open3Dの`compute_fpfh_feature`で取得できます。
引数は、特徴量を計算したい点群と、`search_param`には各店の特徴量を計算する際に使用する周辺の点の設定を渡します。`radius=0.03, max_nn=100`を上げ下げすると、特徴量の値は変わります。



In [ ]:
import sys
import open3d as o3d

# ファイル名の定義，今回はcolab上にダウンロードしたスタンフォードバニーを用います
filename = "bunny/reconstruction/bun_zipper.ply"
print("Loading a point cloud from", filename)
# oprn3dの読み込み関数を利用
pcd = o3d.io.read_point_cloud(filename)
print(pcd)
# 特徴量の計算に使用するため，各点の法線を計算します
pcd.estimate_normals(
    search_param = o3d.geometry.KDTreeSearchParamHybrid(radius=0.01, max_nn=10))

# open3d のfprh特徴量を計算する関数を使用
fpfh = o3d.pipelines.registration.compute_fpfh_feature(pcd,
    search_param = o3d.geometry.KDTreeSearchParamHybrid(radius=0.03, max_nn=100))

print(fpfh)
print(fpfh.data.shape)


## 大域特徴量

### 3D shape histogram

Open3Dに関数が用意されていないので、Numpyで3D shape histogramを実装します。
`getShapeHistogram`が対象の関数で、点群と、ヒストグラムの設定のパラメータを入力すると3D shape histogram特徴量が得られます。

スタンフォードバニーを用いて、実際に計算してみます。
ヒストグラムを可視化した結果において、横軸は各ビン、縦軸は点数を示しています。

In [ ]:
def getShapeHistogram(Ps, Ns, NShells, RMax):
    # ヒストグラム格納用
    hist = np.zeros(NShells)
    # ビンの設定
    bins = np.square(np.linspace(0.0, RMax, NShells + 1))
    indices = np.digitize(np.sum(np.multiply(Ps, Ps), axis=0), bins) - 1
    # ビンごとの点数カウント
    count = np.bincount(indices)[:NShells]
    # 結果を格納
    hist[:count.shape[0]] = count
    return hist

In [ ]:
import matplotlib.pyplot as plt
#スタンフォードバニーを入力する
filename = "bunny/reconstruction/bun_zipper.ply"
print("Loading a point cloud from", filename)
pcd = o3d.io.read_point_cloud(filename)
points = np.asarray(pcd.points).transpose()
# 法線の推定
pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
normals = np.asarray(pcd.normals).transpose()

# 3D Shape Histogramの計算
bins = 30 # histogramのビン数
maxr = 0.3 # 最大の半径をどれくらいにするか
# 3D Shape Histogram関数を用いる
hist = getShapeHistogram(points, normals, bins, maxr)

# 特徴量の可視化
fig = plt.figure()

ax = fig.add_subplot(1, 1, 1)
# x 軸のラベルを設定する。
ax.set_xlabel("bins")
# y 軸のラベルを設定する。
ax.set_ylabel("Number of Points")
x = range(bins)
ax.bar(x, hist)
plt.show()
